# Building a Financial Model

In the refresher notebook which accompanies this one as an introduction to using the 'yfinance' library, I showed how you get financial statements and how to call the functions that return specific data. Now we want to use these financial statements to do some financial modelling.   

As such, this notebook will be the first in a series of two. This piece of work will seek to create a discounted casfhlow model using Apple's financial data. The second series will be to see if this entire process can be sped up by tranforming the code used in this notebook into functions so that one simply needs to run a few commands to pull bespoke outputs.

So let's load our data. We will use Apple's data again just like we did in the refresher. For this we are going to run a config file, that has our ticker specified, as well as the growth rate and the WACC.

In [5]:
import pandas as pd
import yfinance as yf
from yahoofinancials import YahooFinancials

# To get analysts info
import yahoo_fin.stock_info as si

# Required to calculate net present value
import numpy_financial as npf

# For reading properties
from jproperties import Properties


One way to specify some parameters so that the report can be reproduced without having to manually change the overarching code is to store information in a config file. This way, you can change details such as the ticker, without having to worry about the main code. In some places I have preferred to call the ticker manually, and will be something that will need to be addressed when building Part 2 of this project.

In [10]:
configs = Properties()

with open('config/yf_dcf.properties', 'rb') as config_file:
     configs.load(config_file)

TICKER = configs.get('TICKER').data
# Perpetual growth rates is read as a string; we need to convert it to floats.
PERP_GROWTH_RATE = float(configs.get('PERP_GROWTH_RATE').data)
WACC_RATE = float(configs.get('WACC_RATE').data)

# Constants
PROJECTION_YEARS = 5

To get the statements, execute the function that returns the statement data using the respective ticker defined in the config file.

In [24]:
balance_sheet = yf.Ticker(TICKER).balance_sheet
inc_statement = yf.Ticker(TICKER).income_stmt
cf_statement = yf.Ticker(TICKER).cash_flow


The fist issue that we have is that the data is not presented in a way that is familiar to read. To resolve this, we can go to the [SEC website](https://www.sec.gov/ix?doc=/Archives/edgar/data/320193/000032019323000106/aapl-20230930.htm) and fetch Apple's latest 10K.  

From this filing, we can can:
* Crosscheck the values to ensure the data we are pulling is correct for the latest financial year
* Build a structured dataframe where the records (rows) are ordered in the correct sequence  

I want to leave the raw data the way it is in case we would like to use the data later, so let's create a new variable and assign it to the income statement data:


In [8]:
apple_income = inc_statement
inc_statement

,2023-09-30,2022-09-30,2021-09-30,2020-09-30
Tax Effect Of Unusual Items,0.0,0.0,0.0,0.0
Tax Rate For Calcs,0.147192,0.162045,0.133023,0.144282
Normalized EBITDA,125820000000.0,130541000000.0,120233000000.0,77344000000.0
Net Income From Continuing Operation Net Minority Interest,96995000000.0,99803000000.0,94680000000.0,57411000000.0
Reconciled Depreciation,11519000000.0,11104000000.0,11284000000.0,11056000000.0
Reconciled Cost Of Revenue,214137000000.0,223546000000.0,212981000000.0,169559000000.0
EBITDA,125820000000.0,130541000000.0,120233000000.0,77344000000.0
EBIT,114301000000.0,119437000000.0,108949000000.0,66288000000.0
Net Interest Income,NaN,-106000000.0,198000000.0,890000000.0
Interest Expense,NaN,2931000000.0,2645000000.0,2873000000.0


Before we get moving, we quickly just want to look at Apple's income statement from their 10-K and ensure that:
1. We are seeing the correct values
2. We can identify the correct order in which to organise the data  

![alt text](Images/apple_incstatement.png "Title")

We can now see which elements we want in our income statement from the data we have pulled from Yahoo Finance, so let's get to work constructing the income statement. We will call this variable 

In [4]:
inc_structured = apple_income.loc[['Total Revenue', 'Cost Of Revenue', 'Gross Profit', 'Research And Development', 'Selling General And Administration', 'Operating Expense', 'Total Operating Income As Reported', 'Other Non Operating Income Expenses', 'Pretax Income', 'Tax Provision', 'Net Income', 'Basic Average Shares', 'Diluted Average Shares']]
inc_structured

,2023-09-30,2022-09-30,2021-09-30,2020-09-30
Total Revenue,383285000000.0,394328000000.0,365817000000.0,274515000000.0
Cost Of Revenue,214137000000.0,223546000000.0,212981000000.0,169559000000.0
Gross Profit,169148000000.0,170782000000.0,152836000000.0,104956000000.0
Research And Development,29915000000.0,26251000000.0,21914000000.0,18752000000.0
Selling General And Administration,24932000000.0,25094000000.0,21973000000.0,19916000000.0
Operating Expense,54847000000.0,51345000000.0,43887000000.0,38668000000.0
Total Operating Income As Reported,114301000000.0,119437000000.0,108949000000.0,66288000000.0
Other Non Operating Income Expenses,-565000000.0,-334000000.0,258000000.0,803000000.0
Pretax Income,113736000000.0,119103000000.0,109207000000.0,67091000000.0
Tax Provision,16741000000.0,19300000000.0,14527000000.0,9680000000.0


We now have something that looks a lot more like an actual income statement, with our items in the correct order.

## Building the DCF

For this, we want to use both the income statement and the balance sheet.  

For purposes of simplicity, we don't really need to order the data as such, as we just want to pull the values. So we will use our two objects:
* balance_sheet
* inc_statement

The elements we want to use from these two statements are:
* EBIT
* Tax Rate
* Non-cash expenses
* Change in (current assets - current liabilities)
* Capex
* Free Cash Flow

The EBIT we will pull from our income statement data, as well as the tax rate.

In [25]:
ebit = inc_statement.loc['EBIT'][0]
tax_rate = inc_statement.loc['Tax Rate For Calcs'][0]

The change in the working capital we will fetch from the balance sheet. Because we are calculating the change in working capital, we need to first calculate the changes in inventory, receivables, and payables. Thereafter, we simply subtract the receivables and inventory from the accounts payable.

In [ ]:
payable_wc = balance_sheet.loc['Accounts Payable'][0] - balance_sheet.loc['Accounts Payable'][1]
receivable_wc = balance_sheet.loc['Receivables'][0] - balance_sheet.loc['Receivables'][1]
inventory_wc = balance_sheet.loc['Inventory'][0] - balance_sheet.loc['Inventory'][1]
cwc = payable_wc - receivable_wc - inventory_wc

Another way to do this is to wrap these calculations in a function and then run the function on the balance sheet dataset.

In [27]:
def calculate_cwc(balance_sheet):
    # Calculate the current wc
    payable_wc = balance_sheet.loc['Accounts Payable'][0] - balance_sheet.loc['Accounts Payable'][1]
    receivable_wc = balance_sheet.loc['Receivables'][0] - balance_sheet.loc['Receivables'][1]
    inventory_wc = balance_sheet.loc['Inventory'][0] - balance_sheet.loc['Inventory'][1]
    return payable_wc - receivable_wc - inventory_wc

In [38]:
# Change in working capital
cwc = calculate_cwc(balance_sheet)

depreciation_amoritization = cf_statement.loc['Depreciation And Amortization'][0]
cap_exp = cf_statement.loc['Capital Expenditure'][0]

### Calculate unlevered free cashflow.

In [30]:
# Calculate Unlevered Free Cash Flow 
unlevered_fcf = (ebit * (1-tax_rate)) + depreciation_amoritization + cwc + cap_exp

We can then calculate the FFCF, given the growth rate which we can either fetch from Yahoo Finance or calculate as follows:

In [31]:
growth_est_df = si.get_analysts_info(TICKER)['Growth Estimates']
growth_str = growth_est_df[growth_est_df['Growth Estimates'] == 'Next 5 Years (per annum)'][TICKER].iloc[0]
growth_rate = round(float(growth_str.rstrip('%')) / 100.0, 4)

In [40]:
ffcf = []

# Year 1
ffcf.append(unlevered_fcf * (1 + growth_rate))

# Starting from 2nd year
for i in range(1, PROJECTION_YEARS):
    ffcf.append(ffcf[i-1] * (1 + growth_rate))

An easier option is to just fetch the FFCF from yahoo finance itself:

In [44]:
# Free cash flow
free_cash_flow = cf_statement.loc['Free Cash Flow'][0]

ffcf_2 = []
# Year 1
ffcf_2.append(free_cash_flow * (1 + growth_rate))

# Starting from year 2
for i in range(1, PROJECTION_YEARS):
    ffcf_2.append(ffcf[i-1] * (1 + growth_rate))

# Check the value
ffcf_2

[106534963200.00002,
 108833347998.74648,
 116429915689.05899,
 124556723804.15532,
 133250783125.68536]

In [33]:
ffcf

[101732424751.11841,
 108833347998.74648,
 116429915689.05899,
 124556723804.15532,
 133250783125.68536]

As we can see, we do get different values, so I'd leave it to the reader to decide which they believe is best to use.

## Calculating Terminal, Enterprise & Equity Values

### Terminal Value

To calculate the terminal valuem we can use the following calculation:  

$ Terminal Value = \frac{(Free Cash Flow \times [1 + g])}{(WACC – g)}$  

* Remember, we have pulled the growth rate from the config file but it can always be specified in the code.

In [48]:
tv_apple = ffcf[-1] * (1 + PERP_GROWTH_RATE)/(WACC_RATE - PERP_GROWTH_RATE)
tv_apple

31384323809701.164

### Enterprise Value

We calculate the enterprise value by using the following formula:  

$ \text{Enterprise Value} = PV(\text{Free Cash Flow}) + PV(\text{Terminal Value})$  

This can be rewritten as:  

$ Enterprise Value = PV(\text{Free Cash Flow + Terminal Value})$  

In [47]:
# Let's add the terminal value to the last year so the cashflows take this value into account
ffcf[-1] = ffcf[-1] + tv_apple

# We now use the numpy financial library to calculate the net present value that we pass in as an argument
# As well as the WACC that we specified in our config file

# Calculate dcf using npv - add zero or else the method assumes first value as the initial investment
enterprise_value = npf.npv(WACC_RATE, [0] + ffcf)

In [55]:
# Let's take a quick look at where this terminal value was added in our ffcf array:
ffcf

[101732424751.11841,
 108833347998.74648,
 116429915689.05899,
 124556723804.15532,
 133250783125.68536,
 106534963200.00002,
 108833347998.74648,
 116429915689.05899,
 124556723804.15532,
 2112700822311.5908]

### Equity Value  

We use the following formula to calculate the equity value of Apple:  

$ \text{Equity Value = Enterprise Value + Cash and Cash Equivalents + ST and LT Investments} $  

If we take a look at our balance sheet, we can identify what we want, namely:  
* Cash & Cash Equivalents
* Current Debt (and any short term obligations)
* Long-term Debt


In [56]:
balance_sheet

,2023-09-30,2022-09-30,2021-09-30,2020-09-30
Treasury Shares Number,0.0,NaN,NaN,NaN
Ordinary Shares Number,15550061000.0,15943425000.0,16426786000.0,16976763000.0
Share Issued,15550061000.0,15943425000.0,16426786000.0,16976763000.0
Net Debt,81123000000.0,96423000000.0,89779000000.0,74420000000.0
Total Debt,111088000000.0,120069000000.0,124719000000.0,112436000000.0
Tangible Book Value,62146000000.0,50672000000.0,63090000000.0,65339000000.0
Invested Capital,173234000000.0,170741000000.0,187809000000.0,177775000000.0
Working Capital,-1742000000.0,-18577000000.0,9355000000.0,38321000000.0
Net Tangible Assets,62146000000.0,50672000000.0,63090000000.0,65339000000.0
Common Stock Equity,62146000000.0,50672000000.0,63090000000.0,65339000000.0


We can clearly see some values, but there is a lot of information in there, so let us just confirm our values by again refering to [Apple's 10-K](https://www.sec.gov/ix?doc=/Archives/edgar/data/320193/000032019323000106/aapl-20230930.htm). To do this you want to go to Item 8: Financial Statements and Supplementary Data, and then find the "CONSOLIDATED BALANCE SHEETS" on page 30.  

First we look at the asset side:

![alt text](Images/assets.png 'Balance Sheet - Assets')

And now for the liabilities:  

![alt text](Images/liabilities.png 'Balance Sheet - Liabilities')

We can see clearly that there is nothing that is going to catch us out here, so we can cross reference these values with what we've pulled from Yahoo Finance, and assign them to variables.  

Cash & Cash Equivalents adds up but as we can see it not immediately clear what short-term debt is. In the data we pulled we can see that it is larger than the value of 'Term debt'. This is because we need to add the commercial paper owed to the short term debt which would equate to the 'Current Debt' figure we see in the data.  

A quick check will confirm this:

In [58]:
balance_sheet.loc['Current Debt'][0] == (balance_sheet.loc['Other Current Borrowings'][0] + balance_sheet.loc['Commercial Paper'][0])

True

In [57]:
# Calculate Cash And Cash Equivalents
cash_and_equivalents = balance_sheet.loc['Cash And Cash Equivalents'][0]

# Current debt - only interested in the value
current_debt = balance_sheet.loc['Current Debt'][0]

# Long term debt - only interested in the value
long_term_debt = balance_sheet.loc['Long Term Debt'][0]

# Now we just plug and play
equity_value = enterprise_value + cash_and_equivalents - (current_debt + long_term_debt)

### Intrinsic Valuation

All we need to do now is calculate the intrinsic values of the intrinsic value of Apple. For this, we use the following formula:  

$\text{Instrinsic Value} = \frac{\text{Equity Value}}{\text{Shares Outstanding}}$

In [59]:
# For shares outstanding, we can just fetch the latest record from our income statement
shares_outstanding = pd.to_numeric(inc_statement.loc["Basic Average Shares"][0])

# Intrinsic value
intrinsic_value = equity_value / shares_outstanding
intrinsic_value


92.5067481855953

We can make this look slightly better by wrapping it in a few strings. This won't necessarily be crucial for now, but we will do it in part 2 when we try and create a python script that runs the entire code which will return a string with the intrinsic value in resoect to the share price.  

For now we can just do a basic string for completeness. We will first need the current share price which we can fetch live:

In [ ]:
# Current stock price
current_value = si.get_live_price(TICKER)

And now we just wrap the two values together in a string.

In [63]:
'Current Price: ${:.2f} Intrinsic Value: ${:.2f}'.format(round(current_value,2), round(intrinsic_value,2)) + ' for ticker ' + TICKER

'Current Price: $181.82 Intrinsic Value: $92.51 for ticker AAPL'

## Summary & Next Steps  

This notebook introduced how financial modelling works using Python libraries to find the intrinsic value of a stock, in our case, Apple. It took many influences from some great tutorials around the web and a lot of trial and error trying to figure out how to use as few techniques as possible, instead of forcing an answer by combining multiple libraries and complex commands together. That being said, as we move forward it will be necessary to dive deeper into these packages, not just to become more proficient but rather more efficient in terms of drawing insights in a speedy manner. 

The next goal, which will be done in Part 2, will be to write a .py file that combines what we've done in this notebook into a single script. The goal will be to take that script, run it by specifying details within a config file, and return the intrinsic value of the share. Ideally, it would be great to have multiple scripts that use the same technique but different methods (as we have seen with the calculation of the free cashflow), to be able to have a more comprehensive view of what the intrinsic value might be, and to also understand on a deeper level the nuances invovled and how to maximise the potential of the python libraries.